In [1]:
import random
import numpy as np

In [2]:
class BitSet:
    def __init__(self,size):
        self.bits = np.full((1,size), False)

    def get(self, index):
        return self.bits[0,index]

    def set(self,index, value):
        self.bits[0,index] = value

    def flip(self,index):
        self.bits[0,index] = not self.bits[0,index]

    def debug(self):
        print(self.bits)

In [10]:
class Chromossome:
    def __init__(self):
        x = random.randint(-15,15)
        y = random.randint(-15,15)

        self.__genes = Chromossome.get_genotype(x,y)

    @staticmethod
    def get_genotype(x,y):
        bits = BitSet(10)

        xy_binary = "{:05b}".format(x) + "{:05b}".format(y)
        for i in range(10):
            bits.set(i,xy_binary[i] == '1')
        return bits

    @staticmethod
    def get_fenotype(genes):
        # xxxxx yyyyyy genótipo
        # 01234 56789  índice
        # 8421S 8421S  S=0 => + S=1 =>-
        x = (8 * genes.get(0) +
            4 * genes.get(1) +
            2 * genes.get(2) +
            1 * genes.get(3))

        if genes.get(4) == 1:
            x *= -1

        y = (8 * genes.get(5) +
            4 * genes.get(6) +
            2 * genes.get(7) +
            1 * genes.get(8))

        if genes.get(9) == 1:
            y *= -1

        return x,y

    def get_genes(self):
        return self.__genes

    def set_genes(self,genes):
        self.__genes = genes

    def to_string(self):
        chr_str = "G = ["
        for i in range(10):
            if i == 5:
                chr_str += " "
            chr_str += "1" if self.__genes.get(i) else "0"

        x,y = Chromossome.get_fenotype(self.__genes)
        chr_str += f"], F = [{x}, {y}]"
        return chr_str

In [13]:
class Problem:

    @staticmethod
    def f(x,y):
        return x**2 + y**2

    @staticmethod
    def g(x,y):
        return 1/(1 + Problem.f(x,y))

    @staticmethod
    def f_chromossome(chromossome):
        genes = chromossome.get_genes()
        x,y = Chromossome.get_fenotype(genes)
        return Problem.f(x,y)
    @staticmethod
    def g_chromossome(chromossome):
        genes = chromossome.get_genes()
        x,y = Chromossome.get_fenotype(genes)
        return Problem.g(x,y)

    @staticmethod
    def f_average(population):
        sum = 0
        for c in population:
            sum += Problem.f_chromossome(c)
        return sum/ len(population)
    @staticmethod
    def g_average(population):
        sum = 0
        for c in population:
            sum += Problem.g_chromossome(c)
        return sum/ len(population)

In [14]:
class GeneticUtils:

    @staticmethod
    def find_best_chromossome(population):
        best_chromossome = None

        for c in population:
            score = Problem.g_chromossome(c)

            if best_chromossome is None or score > Problem.g_chromossome(best_chromossome):
                best_chromossome = c
        return best_chromossome

    @staticmethod
    def find_worst_chromossome(population):
        worst_chromossome = None

        for c in population:
            score = Problem.g_chromossome(c)

            if worst_chromossome is None or score < Problem.g_chromossome(worst_chromossome):
                best_chromossome = c
        return worst_chromossome

    @staticmethod
    def format_chromossome(chromossome):
        return f"{chromossome.to_string}, Score = {Problem.g_chromossome(chromossome):.3f}"